# Selector Group Chat

SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism.

In [13]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [14]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

In [15]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [16]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100


data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

# Termination Conditions

In [17]:
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_messages_termination

In [18]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

Current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planning agent has assigned task before other agents start working.
Only Select one agent.
'''

In [19]:
'''
Try not to overload the selector prompt with too much information.


selector_prompt (str, optional) – The prompt template to use for selecting the next speaker. 

Available fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. 
1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. 
2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. 

3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.
'''

'\nTry not to overload the selector prompt with too much information.\n\n\nselector_prompt (str, optional) – The prompt template to use for selecting the next speaker. \n\nAvailable fields: ‘{roles}’, ‘{participants}’, and ‘{history}’. \n1. {participants} is the names of candidates for selection. The format is [“<name1>”, “<name2>”, …]. \n2. {roles} is a newline-separated list of names and descriptions of the candidate agents. The format for each line is: “<name> : <description>”. \n\n3. {history} is the conversation history formatted as a double newline separated of names and message content. The format for each message is: “<name> : <message content>”.\n'

In [20]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [21]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [22]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
Here's the plan to break down the task into subtasks: 

1. WebSearchAgent: Find out which Miami Heat player had the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Gather the total rebounds for this player in both the 2007-2008 and 2008-2009 NBA seasons.
3. DataAnalystAgent: Calculate the percentage change in total rebounds for this player between the 2007-2008 and 2008-2009 seasons.

Let's proceed with these tasks.
---------- WebSearchAgent ----------
[FunctionCall(id='call_wo456ReHkDIwQHE0bqft93XO', arguments='{"query":"Miami Heat player highest points 2006-2007 season"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 20

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=165, completion_tokens=113), metadata={}, content="Here's the plan to break down the task into subtasks: \n\n1. WebSearchAgent: Find out which Miami Heat player had the highest points in the 2006-2007 NBA season.\n2. WebSearchAgent: Gather the total rebounds for this player in both the 2007-2008 and 2008-2009 NBA seasons.\n3. DataAnalystAgent: Calculate the percentage change in total rebounds for this player between the 2007-2008 and 2008-2009 seasons.\n\nLet's proceed with these tasks.", type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=267, completion_tokens=27), met

# Custom Selector Function

In [23]:
s = "Mayank"
s[-1]

'k'

In [24]:
planning_agent.name

'PlanningAgent'

In [25]:
from typing import Sequence
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage

def selector_func(messages : Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if(messages[-1].source != planning_agent.name):
        return planning_agent.name
    return None

In [26]:
await selector_team.reset()
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func)

In [27]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To address your query, we will first need to identify the Miami Heat player with the highest points during the 2006-2007 season. Next, we will gather data regarding this player's total rebounds in the 2007-2008 and 2008-2009 seasons to calculate the percentage change.

Here are the tasks broken down:

1. WebSearchAgent : Find the Miami Heat player with the highest points scored in the 2006-2007 NBA season.
2. WebSearchAgent : Gather data on that player's total rebounds in the 2007-2008 NBA season.
3. WebSearchAgent : Gather data on that player's total rebounds in the 2008-2009 NBA season.
4. DataAnalystAgent : Calculate the percentage change in the player's total rebounds from the 2007-2008 season to the 2008-2009 season.
---------- WebSearchAgent

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=165, completion_tokens=180), metadata={}, content="To address your query, we will first need to identify the Miami Heat player with the highest points during the 2006-2007 season. Next, we will gather data regarding this player's total rebounds in the 2007-2008 and 2008-2009 seasons to calculate the percentage change.\n\nHere are the tasks broken down:\n\n1. WebSearchAgent : Find the Miami Heat player with the highest points scored in the 2006-2007 NBA season.\n2. WebSearchAgent : Gather data on that player's total rebounds in the 2007-2008 NBA season.\n3. WebSearchAgent : Gather data on that player's total rebounds in the 2